In [58]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel


from xgboost import XGBRegressor

In [2]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [93]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCoutner
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_pos_rows_transformer import SelectPosRowsTransformer
from transformers.select_features_wrapper import SelectFeaturesWrapper
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_pos_rows_transformer)
importlib.reload(transformers.select_features_wrapper)

import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [4]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/features_filtered.csv', index_col='game_id')

In [5]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([
                     ('get_relavent_games',  SelectRowsTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectPosRowsTransformer('is_player', target=DATA_NAME)),
                    ])

In [6]:
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)

In [7]:
n_data = n_games.merge(n_turns, left_index=True, right_index=True)

In [8]:
featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),                                              
                      ]

In [9]:
reg_pipe = Pipeline(featureTransformers)
t_data = reg_pipe.transform(n_data)

In [10]:
grid = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0),
    'param_grid':
        {
        },
    'scores': ['neg_root_mean_squared_error']
    },
}

In [11]:
# my_searcher = Searcher(reg_pipe, grid, 2, redo_scores=True)
# my_searcher.search(n_data, n_ratings)

In [12]:
# res = SearcherResultsOrgenizer(redo_scores=True).orgenize(my_searcher.get_results())

In [14]:
# res

,,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score
model_name,scorer,,,,,,,,,,,
xgboost,neg_root_mean_squared_error,57.860744,8.760756,1.337651,0.21865,-112.310222,-112.802417,-114.831927,-113.71581,-110.332244,-112.798524,1.503139


In [106]:
# thresholds = my_searcher.get_results()[('xgboost', 'neg_root_mean_squared_error')].best_estimator_[-1].feature_importances_
# thresholds = np.sort(thresholds)
# model =  my_searcher.get_results()[('xgboost', 'neg_root_mean_squared_error')].best_estimator_[-1]


In [24]:
# selection = SelectFromModel(model, threshold=0.1, prefit=True)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00084566, 0.00113591,
       0.00130682, 0.00146642, 0.00162189, 0.00178969, 0.00194677,
       0.0021806 , 0.00262951, 0.00440825, 0.0104104 ], dtype=float32)

In [ ]:
# for thresh in thresholds[::20]:
#  # select features using threshold
#  selection = SelectFromModel(model, threshold=thresh, prefit=True)
#  select_X_train = selection.transform(X_train)
#  # train model
#  selection_model = XGBClassifier()
#  selection_model.fit(select_X_train, y_train)
#  # eval model
#  select_X_test = selection.transform(X_test)
#  predictions = selection_model.predict(select_X_test)
#  accuracy = accuracy_score(y_test, predictions)
#  print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [118]:
s_featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('drops', NameDropperTransformer(['first', 'created_at', 'time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'bot_name'])),
                       ('selection', SelectFromModel(XGBRegressor(random_state=0))),
                       ('xgboost', XGBRegressor(random_state=0))                                          
                      ]

s_reg_pipe = Pipeline(s_featureTransformers)


In [114]:
model.feature_importances_

array([0.02389621, 0.0104104 , 0.00431157, 0.00703815, 0.00671391,
       0.09814946, 0.00156412, 0.00684936, 0.00199115, 0.00528832,
       0.0013184 , 0.00206186, 0.        , 0.00130369, 0.01497081,
       0.00846764, 0.04549474, 0.01041721, 0.        , 0.00167858,
       0.00227139, 0.        , 0.0073297 , 0.00179197, 0.00194404,
       0.00158497, 0.00095647, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [119]:
parameters = {'selection__threshold': thresholds[::20]}

In [120]:
clf = GridSearchCV(s_reg_pipe, parameters, n_jobs=4, scoring='neg_root_mean_squared_error')


In [121]:
clf.fit(n_data, n_ratings)   